In [1]:
import argilla as rg


In [2]:
rg.init(
    api_url="http://localhost:6900",
    # api_key="team.apikey"
)

In [3]:
import os
from json import loads
import openai
from datasets import load_dataset
import pandas as pd
from argilla.metrics.text_classification import f1
from argilla.metrics.commons import text_length

In [4]:
banking_ds = load_dataset("argilla/banking_sentiment_setfit")
# preview dataset content
banking_ds["train"].to_pandas().head(15)

[03/23/23 20:44:59] WARNING  WARNING:datasets.builder:Found cached dataset parquet                   ]8;id=147466;file://C:\Users\DEEPANSHU\Documents\anaconda\envs\argilla_assignment\lib\site-packages\datasets\builder.py\builder.py]8;;\:]8;id=382565;file://C:\Users\DEEPANSHU\Documents\anaconda\envs\argilla_assignment\lib\site-packages\datasets\builder.py#798\798]8;;\
                             (C:/Users/DEEPANSHU/.cache/huggingface/datasets/argilla___parquet/argil               
                             la--banking_sentiment_setfit-4a60f83f113675bf/0.0.0/2a3b91fbd88a2c90d1d               
                             bbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)                                        

  0%|          | 0/2 [00:00<?, ?it/s]

,text,label
0,are you sending me my card?,1
1,Two days ago I did a transfer to another accou...,1
2,Why didn't I receive the right amount of cash?,0
3,Is there a reason why my virtual card won't work?,0
4,Why is my balance the same after a transfer?,1
5,My money hasn't been transferred yet.,0
6,"It is still pending, I am still waiting. Your ...",0
7,What are the places that you deliver cards to?,1
8,Why are my withdrawals suddenly being declined?,1
9,How do I reset my passcode?,1


In [5]:
PROMPT_TEMPLATE = """
Classify the sentiment of the customer request using the following JSON format. Use positive, negative, and neutral in lowercase:

{"prediction": sentiment label string, "explanation": sentence string describing why you think is the sentiment}

Customer request:

"""

In [6]:
records = load_dataset("argilla/banking_sentiment_zs_gpt3", split="train")
# rg.log(records, "banking_sentiment_zs_gpt3")

[03/23/23 20:45:04] WARNING  WARNING:datasets.builder:Found cached dataset parquet                   ]8;id=214955;file://C:\Users\DEEPANSHU\Documents\anaconda\envs\argilla_assignment\lib\site-packages\datasets\builder.py\builder.py]8;;\:]8;id=124547;file://C:\Users\DEEPANSHU\Documents\anaconda\envs\argilla_assignment\lib\site-packages\datasets\builder.py#798\798]8;;\
                             (C:/Users/DEEPANSHU/.cache/huggingface/datasets/argilla___parquet/argil               
                             la--banking_sentiment_zs_gpt3-f7d215e9f2846b59/0.0.0/2a3b91fbd88a2c90d1               
                             dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)                                       

## Copy your api key here.

In [7]:
# set your api key as ENV, for example with Python: os.environ["OPENAI_API_KEY"]
openai.api_key = "COPY HERE"



def classify(text):
    # build prompt with template and input
    prompt = f"{PROMPT_TEMPLATE}\n{text}\n"
    # use create completion template
    completion = openai.Completion.create(
      model="text-davinci-003",
      prompt=prompt,
      temperature=0,
      max_tokens=256
    )
    # get first choice text
    json_response = completion["choices"][0]["text"].strip()
    try:
        prediction = loads(json_response)
    except:
        # for some examples, json is not correctly formatted
        return {"prediction": None, "explanation": f"Wrong JSON format: {json_response}" }
    return prediction

In [8]:
# let's predict over the test set to eval our zero-shot classifier
test_ds_with_preds = [classify(example) for example in banking_ds["test"]['text'] ]

pd.set_option('display.max_colwidth', None)
test_ds_with_preds = pd.DataFrame(test_ds_with_preds)
# test_ds_with_preds.to_pandas().head(15)

In [13]:
test_ds_with_preds

,prediction,explanation
0,negative,"The customer is expressing dissatisfaction with their card payment having the wrong exchange rate, indicating a negative sentiment."
1,neutral,"The customer is expressing a belief, but not expressing any sentiment about the situation."
2,negative,"The customer is expressing frustration and confusion about being charged for getting cash, indicating a negative sentiment."
3,negative,"The customer is expressing frustration and urgency due to the delay in their cash being available, indicating a negative sentiment."
4,neutral,The customer is simply asking a question and does not express any sentiment.
5,neutral,The customer is simply asking a question and does not express any sentiment.
6,negative,"The customer is expressing concern about a fraudulent charge on their account, indicating a negative sentiment."
7,neutral,The customer is asking a question without expressing any sentiment.
8,negative,"The customer is expressing frustration with an issue that has not been resolved, indicating a negative sentiment."
9,negative,"The customer is expressing frustration with the slow processing of their check deposit, indicating a negative sentiment."


In [9]:
# print(banking_ds["test"]['text'][:10])

In [17]:
import argilla as rg

# get the label strings to turn int ids into string names
labels = banking_ds["test"].features["label"].names

records = []
for index, example in test_ds_with_preds.iterrows():
    # create a record with ground-truth annotations and gpt-3 predictions
    record = rg.TextClassificationRecord(
        inputs={"explanation": example["explanation"]},
        # annotation=labels[example["label"]],
        prediction=[(example["prediction"].lower(), 1.0)]
    )
    records.append(record)

# create a dataset in Argilla
rg.log(records, "banking_sentiment_zs_gpt3")

Output()

36 records logged to ]8;id=923912;http://localhost:6900/datasets/argilla/banking_sentiment_zs_gpt3\http://localhost:6900/datasets/argilla/banking_sentiment_zs_gpt3]8;;\

BulkResponse(dataset='banking_sentiment_zs_gpt3', processed=36, failed=0)